# Broiler Weight Estimation - Example Inference

This notebook demonstrates how to use the broiler weight estimation pipeline for inference on RGB and depth image pairs.

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Add project root to path
sys.path.append('..')

from pipeline.pipeline import BroilerWeightPipeline
from pipeline.utils import load_config

## 1. Initialize Pipeline

Load the configuration and initialize the pipeline with trained models.

In [ ]:
# Load configuration
config = load_config('../pipeline/config.yaml')
print("Configuration loaded:")
print(f"- Model artifacts dir: {config['model']['artifacts_dir']}")
print(f"- Default model type: {config['inference']['model_type']}")

In [ ]:
# Initialize pipeline
pipeline = BroilerWeightPipeline('../pipeline/config.yaml')

# Load trained models
try:
    pipeline.load_models()
    print("Models loaded successfully!")
    
    # Display model information
    model_info = pipeline.get_model_info()
    print("\nModel Information:")
    for key, value in model_info.items():
        print(f"- {key}: {value}")
        
except Exception as e:
    print(f"Warning: Could not load models: {e}")
    print("You may need to train models first using: python pipeline/train.py")

## 2. Single Image Inference

Demonstrate inference on a single RGB-Depth image pair.

In [ ]:
# Example paths (update these to your actual data)
rgb_image_path = "../data/rgb/rgb_20250725_162941_594732_instance-0.png"
depth_file_path = "../data/depth/depth_20250725_162941_594732_instance-0.npy"

# Check if example files exist
if os.path.exists(rgb_image_path) and os.path.exists(depth_file_path):
    try:
        # Run inference
        predicted_weight = pipeline.predict_from_images(rgb_image_path, depth_file_path)
        
        print(f"Input RGB image: {rgb_image_path}")
        print(f"Input depth file: {depth_file_path}")
        print(f"Predicted weight: {predicted_weight:.3f} kg")
        
    except Exception as e:
        print(f"Inference failed: {e}")
        
else:
    print("Example data files not found. Update the paths above to your actual data.")
    print(f"Looking for RGB: {rgb_image_path}")
    print(f"Looking for Depth: {depth_file_path}")

## 3. Batch Inference

Run inference on multiple image pairs from directories.

In [ ]:
# Example directories (update these to your actual data)
rgb_directory = "../data/rgb"
depth_directory = "../data/depth"

# Check if directories exist
if os.path.exists(rgb_directory) and os.path.exists(depth_directory):
    try:
        # Run batch inference
        results = pipeline.predict_from_directory(rgb_directory, depth_directory)
        
        print(f"Processed {len(results)} image pairs")
        print(f"Mean predicted weight: {results['predicted_weight_kg'].mean():.3f} kg")
        print(f"Weight range: {results['predicted_weight_kg'].min():.3f} - {results['predicted_weight_kg'].max():.3f} kg")
        
        # Display first few results
        print("\nFirst 10 predictions:")
        display(results.head(10))
        
    except Exception as e:
        print(f"Batch inference failed: {e}")
        
else:
    print("Example data directories not found. Update the paths above to your actual data.")
    print(f"Looking for RGB dir: {rgb_directory}")
    print(f"Looking for Depth dir: {depth_directory}")

## 4. Visualize Results

Create visualizations of the prediction results.

In [ ]:
# Only run if we have results from batch inference
if 'results' in locals() and len(results) > 0:
    # Plot histogram of predicted weights
    plt.figure(figsize=(10, 6))
    
    plt.subplot(1, 2, 1)
    plt.hist(results['predicted_weight_kg'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    plt.xlabel('Predicted Weight (kg)')
    plt.ylabel('Frequency')
    plt.title('Distribution of Predicted Weights')
    plt.grid(True, alpha=0.3)
    
    # Plot time series if we can extract timestamps
    plt.subplot(1, 2, 2)
    plt.plot(range(len(results)), results['predicted_weight_kg'], 'o-', alpha=0.7)
    plt.xlabel('Sample Index')
    plt.ylabel('Predicted Weight (kg)')
    plt.title('Predicted Weights by Sample')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\nSummary Statistics:")
    print(results['predicted_weight_kg'].describe())
    
else:
    print("No results available for visualization. Run batch inference first.")

## 5. Feature Analysis

Examine the features extracted from a sample image pair.

In [ ]:
# Extract features from a sample to understand the feature vector
if os.path.exists(rgb_image_path) and os.path.exists(depth_file_path):
    from pipeline.data_loader import ImagePairDataLoader
    
    # Load and preprocess the image pair
    data_loader = ImagePairDataLoader(os.path.dirname(rgb_image_path), 
                                     os.path.dirname(depth_file_path), 
                                     config)
    
    rgb_image = data_loader.load_rgb_image(rgb_image_path)
    depth_data = data_loader.load_depth_data(depth_file_path)
    
    # Extract individual feature types
    features_2d = pipeline.extractor_2d.extract(rgb_image, rgb_image_path)
    features_3d = pipeline.extractor_3d.extract(depth_data, depth_file_path)
    features_resnet = pipeline.extractor_resnet.extract(rgb_image)
    
    print("Feature Analysis:")
    print(f"2D Features ({len(features_2d)}):")
    for name, value in features_2d.items():
        print(f"  {name}: {value:.4f}")
    
    print(f"\n3D Features ({len(features_3d)}):")
    for name, value in features_3d.items():
        print(f"  {name}: {value:.4f}")
    
    print(f"\nResNet Features: {features_resnet.shape[0]} dimensions")
    print(f"ResNet feature stats - Mean: {np.mean(features_resnet):.4f}, Std: {np.std(features_resnet):.4f}")
    
    # Show feature dimensions
    feature_dims = pipeline.feature_fusion.get_feature_dimensions()
    print(f"\nFeature Dimensions:")
    for feature_type, dim in feature_dims.items():
        print(f"  {feature_type}: {dim}")
        
else:
    print("Sample data not available for feature analysis.")

## 6. Model Comparison

Compare predictions from different models if multiple are available.

In [ ]:
# Compare different models if available
if pipeline.is_trained and os.path.exists(rgb_image_path) and os.path.exists(depth_file_path):
    model_info = pipeline.get_model_info()
    available_models = [m.replace('_model', '') for m in model_info.get('available_models', [])]
    
    if len(available_models) > 1:
        print("Comparing model predictions:")
        
        predictions = {}
        for model_type in available_models:
            try:
                pred = pipeline.predict_from_images(rgb_image_path, depth_file_path, model_type)
                predictions[model_type] = pred
                print(f"  {model_type.upper()}: {pred:.3f} kg")
            except Exception as e:
                print(f"  {model_type.upper()}: Failed ({e})")
        
        if len(predictions) > 1:
            # Calculate differences
            pred_values = list(predictions.values())
            max_diff = max(pred_values) - min(pred_values)
            print(f"\nMaximum difference: {max_diff:.3f} kg")
            print(f"Mean prediction: {np.mean(pred_values):.3f} kg")
    else:
        print(f"Only one model available: {available_models[0] if available_models else 'None'}")
        
else:
    print("Model comparison not available (models not loaded or sample data missing).")

## 7. Export Results

Save prediction results to files for further analysis.

In [ ]:
# Save results if we have them
if 'results' in locals() and len(results) > 0:
    output_file = "prediction_results.csv"
    results.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
    
    # Also save summary statistics
    summary_file = "prediction_summary.txt"
    with open(summary_file, 'w') as f:
        f.write("Broiler Weight Prediction Summary\n")
        f.write("=" * 35 + "\n\n")
        f.write(f"Total samples: {len(results)}\n")
        f.write(f"Mean weight: {results['predicted_weight_kg'].mean():.3f} kg\n")
        f.write(f"Median weight: {results['predicted_weight_kg'].median():.3f} kg\n")
        f.write(f"Min weight: {results['predicted_weight_kg'].min():.3f} kg\n")
        f.write(f"Max weight: {results['predicted_weight_kg'].max():.3f} kg\n")
        f.write(f"Std deviation: {results['predicted_weight_kg'].std():.3f} kg\n")
        
    print(f"Summary saved to {summary_file}")
    
else:
    print("No results to export.")

## Conclusion

This notebook demonstrated:
1. Loading the broiler weight estimation pipeline
2. Running inference on single and multiple image pairs
3. Analyzing features and model outputs
4. Visualizing and exporting results

To use this pipeline with your own data:
1. Update the file paths to point to your RGB and depth data
2. Ensure models are trained using `python pipeline/train.py`
3. Adjust configuration in `pipeline/config.yaml` as needed

For more information, see the pipeline documentation and README files.